In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
img_rows, img_cols = 28, 28
num_classes = 10

def data_prep(raw):
    out_y = to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

In [4]:
train_file = "/kaggle/input/digit-recognizer/train.csv"
raw_data = pd.read_csv(train_file)

x, y = data_prep(raw_data)

In [5]:
x.shape, y.shape

((42000, 28, 28, 1), (42000, 10))

In [6]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
model.fit(x, y,
          batch_size=128,
          epochs=2,
          validation_split = 0.2)

Epoch 1/2
263/263 ━━━━━━━━━━━━━━━━━━━━ 21s 74ms/step - accuracy: 0.8614 - loss: 0.4536 - val_accuracy: 0.9757 - val_loss: 0.0769
Epoch 2/2
263/263 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - accuracy: 0.9815 - loss: 0.0605 - val_accuracy: 0.9774 - val_loss: 0.0728


In [9]:
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
X_test = test.values.reshape(-1, 28, 28, 1) / 255.0

In [10]:
y_pred = np.argmax(model.predict(X_test), axis=1)

# Save submission
submission = pd.DataFrame({'ImageId': np.arange(1, len(y_pred) + 1), 'Label': y_pred})
submission.to_csv('submission.csv', index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
